In [3]:
%reload_ext autoreload
%autoreload 2

In [4]:
import os
import sys
import shutil
import pprint

import pymongo
import json
import pandas as pd

import climatools.lblnew.setup_bestfit as setup_bestfit
import climatools.lblnew.bestfit_params as bestfit
import climatools.lblnew.pipeline as pipeline
import climatools.lblnew.runrecord_bestfit as runrecord

# Run lblnew-bestfit

In [5]:
def get_params():
    '''
    Looks like 7 g-intervals aren't good enough.  Let's try 9.
    '''
    molecule = 'co2'
    band = '3b'
    
    commitnumber = 'a22ab94'
    ng_refs = [5, 4]
    option_compute_ktable = 0
    option_compute_btable = 0
    w_diffuse = [(1.66, 1.66, 1.66, 1.66, 1.66), 
                 (1.66, 1.66, 1.66, 1.66)]
    wgt = [(0, .5, .4, .6, .65), (.7, .7, .55, .6)]
    
    concs = [400e-6, 800e-6]
    atmpros = [('mls', 294), ('saw', 257), ('trp', 300)]
    
    params = []
    for conc in concs:
        for atmpro, tsfc in atmpros:
            param = bestfit.kdist_params(molecule=molecule, band=band)
            param['commitnumber'] = commitnumber
            param['ng_refs'] = ng_refs
            param['option_compute_ktable'] = option_compute_ktable
            param['option_compute_btable'] = option_compute_btable
            param['w_diffuse'] = w_diffuse
            param['wgt'] = wgt
            param['conc'] = conc
            param['atmpro'] = atmpro
            param['tsfc'] = tsfc
            params.append(param)
            
            if atmpro == 'mls' and conc == 400e-6:
                param = param.copy()
                param['option_compute_ktable'] = 1
                param['option_compute_btable'] = 1
                params.append(param)
                              
    return params

In [8]:
params = get_params()

In [9]:
pipeline.pipeline_fortran_bsub(params=params, setup=setup_bestfit)

In [22]:
!bjobs

No unfinished job found


# Save output data to mongodb

In [16]:
client = pymongo.MongoClient('localhost', 27017)

In [42]:
def make_doc(param=None):
    '''
    Makes a Mongodb document for an lblnew-bestfit run.
    
    Parameters
    ----------
    param: dict
        lblnew-bestfit input parameters.
    doc: dict
        lblnew-bestfit input (param) and output (flux and cooling rate)
    '''
    dir_fortran = pipeline.get_dir_case(param=param, setup=setup_bestfit)
    
    fnames = [n for n in os.listdir(dir_fortran) if n in 
              ('output_wfluxg.dat', 'output_wcoolrg.dat',
               'output_fluxg.dat', 'output_coolrg.dat',
               'output_flux.dat', 'output_coolr.dat',
               'abscom.dat', 'dgdgs.dat',
               'kg_lin.dat', 'kg_nonlin.dat')]
    
    s_outputs = []
    for fname in fnames:
        with open(os.path.join(dir_fortran, fname), 
                  mode='r', encoding='utf-8') as f:
            s = f.read()
        
        s_outputs.append(s)
    
    doc = {}
    doc['param'] = param
    
    for fname, s in zip(fnames, s_outputs):
        doc[fname.split('.')[0]] = s
    
    return doc


def make_query(param=None):
    '''
    Returns the MongoDB query for a lblnew-bestfit 
    run's document.
    
    Parameters
    ----------
    param: dict
    lblnew-bestfit input parameters.
    '''
    return {'param.' + name: value for name, value in param.items()}

In [37]:
docs = (make_doc(param=param) for param in [params[1]])

In [38]:
client.lblnew.bestfit_lw.insert_many(docs)

In [74]:
#for param in params:
#    client.lblnew.bestfit_lw.delete_one(make_query(param))

# Run analysis notebook

In [25]:
gprocs = pipeline.pipeline_ipynb2git(params=[params[1]], 
                                     setup=setup_bestfit)

co2 3b a22ab94
co2 3b a22ab94



# Display nbviewer links

In [26]:
for param in params:
    print(param['molecule'], 'band', param['band'], param['atmpro'])
    print(pipeline.nbviewer_url(param=param, setup=setup_bestfit))
    print()

co2 band 3b mls
http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/co2/conc_0.0004/band03b_wn_620_720/nv_1000/dv_0.001/ng_9/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250/ng_refs_5__4/ng_adju_0__0/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0_0.5_0.4_0.6_0.65__0.7_0.7_0.55_0.6/wgt_flux_1/w_diffuse_1.66_1.66_1.66_1.66_1.66__1.66_1.66_1.66_1.66/option_compute_ktable_0/option_compute_btable_0/crd_a22ab94/results.ipynb

co2 band 3b mls
http://nbviewer.jupyter.org/github/qap/analysis_-_new_kdist_param/blob/master/lblnew/co2/conc_0.0004/band03b_wn_620_720/nv_1000/dv_0.001/ng_9/g_ascending_k_descending/refPTs_P_1_T_250__P_10_T_250/ng_refs_5__4/ng_adju_0__0/getabsth_auto__auto/absth_dlogN_uniform__dlogN_uniform/klin_none/atmpro_mls/wgt_k_1/wgt_0_0.5_0.4_0.6_0.65__0.7_0.7_0.55_0.6/wgt_flux_1/w_diffuse_1.66_1.66_1.66_1.66_1.66__1.66_1.66_1.66_1.66/option_compute_ktable_1/option_compute_btable_1/crd_a22ab94/resul